# Course 6 Lab 8 - Access DB2 on cloud using python

In [2]:
# libraries
# !pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3 --user
# !pip install sqlalchemy==1.4.4 --user
# !pip install ipython-sql
import ibm_db

In [3]:
# library for the secret database stuff
# !pip install python-dotenv
from dotenv import load_dotenv
import os
# reads the env file in our repository
load_dotenv()

True

In [4]:
# database credentials from DB2
dsn_hostname = os.environ.get("HOSTNAME")     
dsn_uid = os.environ.get("USERNAME")      
dsn_pwd = os.environ.get("PASSWORD")
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"           
dsn_port =  "31249" 
dsn_protocol = "TCPIP"          
dsn_security = "SSL" 

In [5]:
#Create database connection
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  BLUDB


In [6]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table INSTRUCTOR"

#Now execute the drop statment
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

don't worry about the error above, that is what was supposed to happen, it just means the instructor table does not exist already which is what we want

In [7]:
#Construct the Create Table DDL statement - replace the ... with rest of the statement
createQuery = "create table INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))"

#Now fill in the name of the method and execute the statement
createStmt = ibm_db.exec_immediate(conn,createQuery)

In [8]:
#Construct the query - replace ... with the insert statement
insertQuery = "INSERT INTO INSTRUCTOR VALUES (1,'Rav', 'Ahuja', 'TORONTO', 'CA'), (2, 'Raul', 'Chong', 'Markham','CA'), (3, 'Hima', 'Vasudevan','Chicago','US');"

#execute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [9]:
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only) - replace ... with your code
ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [10]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
   print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

 ID: 2  FNAME: Raul
 ID: 3  FNAME: Hima


In [11]:
#now write and execute an update statement that changes the Rav's CITY to MOOSETOWN
UpdateQuery = "UPDATE INSTRUCTOR SET CITY = 'Moosetown' WHERE FNAME = 'Rav'"
updateStmt = ibm_db.exec_immediate(conn, UpdateQuery)


### Retrieve data into pandas

In [12]:
import pandas
import ibm_db_dbi

In [13]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [14]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf.LNAME[0]

'Ahuja'

In [15]:
#print the entire data frame
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,Moosetown,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [16]:
# use the shape method to see how many rows and columns are in the dataframe
pdf.shape

(3, 5)

In [17]:
# close the connection
ibm_db.close(conn)

True